In [57]:
from shared import const
from shared import utils
import numpy as np
import pandas as pd
import re

# Gensim
import gensim
from gensim import models
import gensim.corpora as corpora
from gensim.utils import simple_preprocess, lemmatize
from gensim.models import CoherenceModel, LdaModel, LdaMulticore

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
#matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

#Choose Random forest for multiclass classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [44]:
#prepare stopwords
from nltk.corpus import stopwords

stop_words = stopwords.words("english")

#for the purpose of emotion classification, we trim the words by extending stopwords:
stop_words.extend(["from", "re", "use", "day", "let", "get","say", "know", "think", "love"])

In [46]:
#preproccesing

#remove words fewer than 3 characters
def remove_words(list2d_words, threshold = 3):
    '''
    return a 2d list of words
    '''
    words_more_than_3chars = []
    for i in range(len(list2d_words)):
        proc_sentence = []
        for j in range(len(list2d_words[i])):
            if len(list2d_words[i][j]) >= threshold:
                proc_sentence.append(list2d_words[i][j])
        words_more_than_3chars.append(proc_sentence)
    return words_more_than_3chars
        

#remove stopwords
#simple_preprocess will remove punctuations and unnecessary characters altogether
def remove_stopwords(list2d_words):
    return [[word for word in simple_preprocess(" ".join(doc)) if word not in stop_words] for doc in list2d_words]

#lemmatize: word in third person changed to first person, etc, and only keep the nouns, adj, verb, adv
nlp = spacy.load('en', disable = ['parse', 'ner'])

def my_lemmatize(list2d_words, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in list2d_words:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [47]:
#tokenize each document into a list of words, removing punctuations and unnecessary characters altogether.
def sentences_to_words(sentences):
    yield(gensim.utils.simple_preprocess(str(sentences), deacc=True))  # deacc=True removes punctuations

import itertools

def corpus_to_2d_words(corpus):
    word_corpus = []
    for i in range(len(corpus)):
        data_words = list(sentences_to_words(corpus[i]))
        word_corpus = [*word_corpus, *data_words]
    return word_corpus

def words_2d_to_dict(word_2d_list):
    dictionary = corpora.Dictionary(word_2d_list)
    return dictionary

def df_to_lemmatized_words(df):
    words_2d = corpus_to_2d_words(df)
    words_morethan_3chars = remove_words(words_2d)
    words_nostops = remove_stopwords(words_morethan_3chars)
    lemmatized_words = my_lemmatize(words_nostops)
    return lemmatized_words

def lemmatized_to_corpus(words_2d):
    dict_words = words_2d_to_dict(words_2d)
    corpus = [dict_words.doc2bow(simple_preprocess(" ".join(line))) for line in words_2d]
    
def build_LDA_model(dict_words, corpus):
    lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                          id2word = dict_words,
                                          num_topics = NUM_TOPICS, #tried 3, 4, 5, 6, 10 , 20. seems 3 for coherence is the best, the second is 4
                                          random_state = 100, #the seed 
                                          update_every = 0, #batch mode
                                          chunksize = 1000, #larger the heavier workload for the memory
                                          passes = 10,#how to determine the chunksize and passes?relevant to size of dataset?
                                          alpha = 'auto',
                                          minimum_probability = 0.0,
                                          per_word_topics = False #non-indicative words would be omitted
                                          )
    return lda_model
    
#evaluate by perplexity and coherence
def print_coherence_perplexity(the_model, lemmatized_words, dict_words, corpus, f=None):
    coherence_model = CoherenceModel(model = the_model, texts = lemmatized_words, dictionary = dict_words, coherence = 'c_v')
    coherence_lda = coherence_model.get_coherence()
    #compute model coherence: higher the better:
    print('\nCoherence Score: ', coherence_lda, file = f)
    #compute model perplexity: lower the better:
    print('\nPerplexity: ', the_model.log_perplexity(corpus), file = f)
    
#get the LDA generated topic-document dataset and responding label for classification
def generate_data(model, corpus):
    topics_features = []
    for i in range(TRAIN_SAMPLE):
        row = []
        for j in range(NUM_TOPICS):
            row.append(model.get_document_topics(corpus[i])[j][1])
        topics_features.append(row)
    #generate data for supervised classification
    gen_df_mid = pd.DataFrame(topics_features)
    return gen_df_mid

def generate_train_test(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
    return X_train, X_test, y_train, y_test

#z-score normalization
def zscore_normalization(X_train, X_test):
    zscoreScaler = StandardScaler().fit(X_train)
    X_tr_std = zscoreScaler.transform(X_train)
    X_ts_std = zscoreScaler.transform(X_test)
    return X_tr_std, X_ts_std

def RF_classify_score(X_tr_std, X_ts_std, y_train, y_test):
    rf_clf = RandomForestClassifier()
    rf_clf.fit(X_tr_std, y_train)
    rf_pred = rf_clf.predict(X_ts_std)
    score = accuracy_score(y_test, rf_pred)
    print(score)
    return score

def complement_classify_score(X_train, X_test, y_train, y_test):
    cnb = ComplementNB(alpha = 0.1)
    cnb.fit(X_train, y_train)
    cnb_pred = cnb.predict(X_test)
    score = accuracy_score(y_test, cnb_pred)
    print(score)
    return score  
    
def randomForest_LDA(X, Y):
    X1_train, X1_test, y1_train, y1_test = generate_train_test(X,Y)
    #z-score normalization
    X1_tr_std, X1_ts_std = zscore_normalization(X1_train, X1_test)
    RF_classify_score(X1_tr_std, X1_ts_std, y1_train, y1_test)
    
def complementNB_LDA(X, Y):
    X_train, X_test, y_train, y_test = generate_train_test(X,Y)
    #X_tr_std, X_ts_std = zscore_normalization(X_train, X_test)
    complement_classify_socre(X_train, X_test, y_train, y_test)


In [48]:
# Create the TF-IDF model
def build_tfidf_model(corpus):
    tfidf = models.TfidfModel(corpus, smartirs='ntc')
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf

In [ ]:
 #use k-fold validation:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
kFolder = KFold(n_splits = 10, shuffle = True) #10-fold cross-validation

def kfold_classification(X, Y, clf, normalization_func = None, f = None):
    sum_accuracy = []
    for k, (train, test) in enumerate(kFolder.split(X, Y)):
        X_train = np.array(X.iloc[train, :])
        X_test = np.array(X.iloc[test, :])
        y_train = np.array(Y.iloc[train])
        #print(y_train.shape)
        y_test = np.array(Y.iloc[test])
        if normalization_func != None:
            X_train, X_test = normalization_func(X_train, X_test)
        #rf_clf = RandomForestClassifier()
        clf.fit(X_train, y_train)
        rf_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, rf_pred)
        sum_accuracy.append(accuracy)
        #print("the {} th fold split got accuracy {}; f1-score: {} ".format(k, accuracy, f1_score(y_test, rf_pred)), file = f)
    k_fold_avg = sum(sum_accuracy)/len(sum_accuracy)
    print("the average of 10 rounds is {}".format(k_fold_avg), file = f)
    return k_fold_avg

In [65]:
data_file = [const.CLEAN_UNEVEN_SPOTIFY, const.CLEAN_UNEVEN_DEEZER, const.CLEAN_SPOTIFY, const.CLEAN_DEEZER]
for every in data_file:
    name = str(every).split('/')[-1].split('.')[0]
    f = open('test_'+ name + '.txt', 'a')
    f.write('this is my name '+ name)
    f.close()

In [66]:


#read data
data_file = [const.CLEAN_UNEVEN_SPOTIFY, const.CLEAN_UNEVEN_DEEZER, const.CLEAN_SPOTIFY, const.CLEAN_DEEZER]
for every in data_file:
    df_full = pd.read_csv(every)
    print(df_full.shape)
    df_1 = utils.get_class_based_data(df_full, 1) #
    df_2 = utils.get_class_based_data(df_full, 2)
    df_3 = utils.get_class_based_data(df_full, 3)
    df_4 = utils.get_class_based_data(df_full, 4)
    df_1.reset_index()
    df_2.reset_index()
    df_3.reset_index()
    df_4.reset_index()
    TRAIN_SAMPLE = int(len(df_1))
    Y1 = df_1.iloc[:TRAIN_SAMPLE, -1]
    Y2 = df_2.iloc[:TRAIN_SAMPLE, -1]
    Y3 = df_3.iloc[:TRAIN_SAMPLE, -1]
    Y4 = df_4.iloc[:TRAIN_SAMPLE, -1]
    train_dflist_1 = df_1.lyrics[:TRAIN_SAMPLE].tolist()
    train_dflist_2 = df_2.lyrics[:TRAIN_SAMPLE].tolist()
    train_dflist_3 = df_3.lyrics[:TRAIN_SAMPLE].tolist()
    train_dflist_4 = df_4.lyrics[:TRAIN_SAMPLE].tolist()

    lemmatized_words_1 = df_to_lemmatized_words(train_dflist_1)
    lemmatized_words_2 = df_to_lemmatized_words(train_dflist_2)
    lemmatized_words_3 = df_to_lemmatized_words(train_dflist_3)
    lemmatized_words_4 = df_to_lemmatized_words(train_dflist_4)

    #print(lemmatized_words)

    #turn the 2d word list to dict 
    dict_words_1 = words_2d_to_dict(lemmatized_words_1)
    dict_words_2 = words_2d_to_dict(lemmatized_words_2)
    dict_words_3 = words_2d_to_dict(lemmatized_words_3)
    dict_words_4 = words_2d_to_dict(lemmatized_words_4)
    #print(dict_words)

    #there's 2 input for lDA: dictinary and the corpus
    #create bag of words
    corpus_1 =  [dict_words_1.doc2bow(simple_preprocess(" ".join(line))) for line in lemmatized_words_1]
    corpus_2 =  [dict_words_2.doc2bow(simple_preprocess(" ".join(line))) for line in lemmatized_words_2]
    corpus_3 =  [dict_words_3.doc2bow(simple_preprocess(" ".join(line))) for line in lemmatized_words_3]
    corpus_4 =  [dict_words_4.doc2bow(simple_preprocess(" ".join(line))) for line in lemmatized_words_4]

    name = str(every).split('/')[-1].split('.')[0]
    f = open('knn_'+ name +'count.txt', 'a')
    #highest score is gotten when number of topics is 6.
    NUM_TOPICS = 6
    for i in range(2, 20):
        n_neighbors = i
        lda_model_1 = build_LDA_model(dict_words_1, corpus_1)
        lda_model_2 = build_LDA_model(dict_words_2, corpus_2)

        lda_model_3 = build_LDA_model(dict_words_3, corpus_3)

        lda_model_4 = build_LDA_model(dict_words_4, corpus_4)
        print("---------------------start of topic # {}".format(i), "-------------------", file = f)
   
        gen_df_mid_1 = generate_data(lda_model_1, corpus_1)
        gen_df_mid_2 = generate_data(lda_model_2, corpus_2)
        gen_df_mid_3 = generate_data(lda_model_3, corpus_3)
        gen_df_mid_4 = generate_data(lda_model_4, corpus_4)

        #randomForest_LDA(gen_df_mid_1, df_1.iloc[:TRAIN_SAMPLE, -1])

        #kfold for class 1:
        print("KNN classificaton:")
        knn_clf = KNeighborsClassifier(n_neighbors)
        knn_score_sum = kfold_classification(gen_df_mid_1, Y1, knn_clf, zscore_normalization)
    
        #randomForest_LDA(gen_df_mid_2, df_2.iloc[:TRAIN_SAMPLE, -1])
        #kfold for class 2:
  
        knn_clf = KNeighborsClassifier(n_neighbors)
        knn_score_sum = knn_score_sum + kfold_classification(gen_df_mid_2, Y2, knn_clf, zscore_normalization)
    
    
        #randomForest_LDA(gen_df_mid_3, df_3.iloc[:TRAIN_SAMPLE, -1])
        #kfold for class 3:
        #random forest classificaton:
        knn_clf = KNeighborsClassifier(n_neighbors)
        knn_score_sum = knn_score_sum + kfold_classification(gen_df_mid_3, Y3, knn_clf, zscore_normalization)
    
    
        #randomForest_LDA(gen_df_mid_4, df_4.iloc[:TRAIN_SAMPLE, -1])
        #kfold for class 4:
        #random forest classificaton:
        rf_clf = RandomForestClassifier()
        knn_clf = KNeighborsClassifier(n_neighbors)
        knn_score_sum = knn_score_sum + kfold_classification(gen_df_mid_4, Y4, knn_clf, zscore_normalization)
    
        knn_score_avg = knn_score_sum / 4
        print("topics: {}, KNN score average of 4 models: {}".format(i, knn_score_avg), file = f)
   
    f.close()

(5313, 6)
KNN classificaton:
the average of 10 rounds is 0.5173291925465839
the average of 10 rounds is 0.521614906832298
the average of 10 rounds is 0.5738302277432712
the average of 10 rounds is 0.5247412008281573
KNN classificaton:
the average of 10 rounds is 0.5189648033126294
the average of 10 rounds is 0.5289026915113872
the average of 10 rounds is 0.5464389233954451
the average of 10 rounds is 0.5650517598343685
KNN classificaton:
the average of 10 rounds is 0.518592132505176
the average of 10 rounds is 0.5433540372670808
the average of 10 rounds is 0.5823809523809523
the average of 10 rounds is 0.5273706004140786
KNN classificaton:
the average of 10 rounds is 0.5043271221532091
the average of 10 rounds is 0.538985507246377
the average of 10 rounds is 0.6010973084886129
the average of 10 rounds is 0.518840579710145
KNN classificaton:
the average of 10 rounds is 0.5303933747412009
the average of 10 rounds is 0.556335403726708
the average of 10 rounds is 0.5723188405797102
the ave

the average of 10 rounds is 0.5145454545454545
KNN classificaton:
the average of 10 rounds is 0.5409090909090909
the average of 10 rounds is 0.5199999999999999
the average of 10 rounds is 0.5681818181818181
the average of 10 rounds is 0.5036363636363637
KNN classificaton:
the average of 10 rounds is 0.5390909090909091
the average of 10 rounds is 0.5009090909090909
the average of 10 rounds is 0.5690909090909091
the average of 10 rounds is 0.5281818181818181
KNN classificaton:
the average of 10 rounds is 0.5345454545454544
the average of 10 rounds is 0.5399999999999998
the average of 10 rounds is 0.5599999999999999
the average of 10 rounds is 0.5027272727272727
KNN classificaton:
the average of 10 rounds is 0.5463636363636363
the average of 10 rounds is 0.5372727272727271
the average of 10 rounds is 0.5654545454545454
the average of 10 rounds is 0.509090909090909
KNN classificaton:
the average of 10 rounds is 0.5463636363636364
the average of 10 rounds is 0.5199999999999999
the average o

Several parameters to adjust:

the number of topics;

alpha and beta: hyperparameter that affect sparsity of the topics. Default to 1.0/num_topics

chunksize: the number of documents to be used in each training chunk

update_every: how often the model parameters should be updated

passes: the total number of training passes

In [53]:
# # Show the Word Weights in Corpus
# num_row = 0
# for doc in corpus:#every row in corpus: id of every word in the doc and the freq of that word in the doc
#     num_row += 1
#     print([[dict_words[id], freq] for id, freq in doc])
#     print("\n{}".format(num_row),"---------------------------------")

#Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

In [54]:
#Build LDA model
#NUM_TOPICS = 8
f = open('uneven_311_spotify_count.txt', 'a')
for i in range(3, 8):
    NUM_TOPICS = i
    lda_model_1 = build_LDA_model(dict_words_1, corpus_1)
    lda_model_2 = build_LDA_model(dict_words_2, corpus_2)

    lda_model_3 = build_LDA_model(dict_words_3, corpus_3)

    lda_model_4 = build_LDA_model(dict_words_4, corpus_4)
    print("---------------------start of topic # {}".format(i), "-------------------", file = f)
    print_coherence_perplexity(lda_model_1, lemmatized_words_1, dict_words_1, corpus_1)
    
    print_coherence_perplexity(lda_model_2, lemmatized_words_2, dict_words_2, corpus_2)

    print_coherence_perplexity(lda_model_3, lemmatized_words_3, dict_words_3, corpus_3)

    print_coherence_perplexity(lda_model_4, lemmatized_words_4, dict_words_4, corpus_4)

    gen_df_mid_1 = generate_data(lda_model_1, corpus_1)
    gen_df_mid_2 = generate_data(lda_model_2, corpus_2)
    gen_df_mid_3 = generate_data(lda_model_3, corpus_3)
    gen_df_mid_4 = generate_data(lda_model_4, corpus_4)

    #randomForest_LDA(gen_df_mid_1, df_1.iloc[:TRAIN_SAMPLE, -1])

    #kfold for class 1:
    print("random forest classificaton:")
    rf_clf = RandomForestClassifier()
    rf_score_sum = kfold_classification(gen_df_mid_1, Y1, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_2, df_2.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 2:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_2, Y2, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_3, df_3.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 3:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_3, Y3, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_4, df_4.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 4:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_4, Y4, rf_clf, zscore_normalization)
    rf_score_avg = rf_score_sum / 4
    print("topics: {}, RF score average of 4 models: {}".format(i, rf_score_avg), file = f)

    print("complement naive bayes classifier:")
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = kfold_classification(gen_df_mid_1, Y1, cnb_clf)
    
    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_2, Y2, cnb_clf)

    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_3, Y3, cnb_clf)
    
    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_4, Y4, cnb_clf)
    cnb_score_avg = cnb_score_sum / 4
    print("Complement NB score average of 4 models: {}".format(cnb_score_avg), file = f)
    print("---------------------end of topic # {}".format(i), "-------------------", file = f)
    f.flush()
f.close()


Coherence Score:  0.3309309385862764

Perplexity:  -7.138909007457849

Coherence Score:  0.3591453409738796

Perplexity:  -7.175866333328538

Coherence Score:  0.32109315609632333

Perplexity:  -7.163283573402322

Coherence Score:  0.3060971054262153

Perplexity:  -7.169672419296917
random forest classificaton:
the average of 10 rounds is 0.5316149068322981


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


the average of 10 rounds is 0.5028364389233955
the average of 10 rounds is 0.5867494824016563


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


the average of 10 rounds is 0.556231884057971
complement naive bayes classifier:
the average of 10 rounds is 0.5042857142857142
the average of 10 rounds is 0.4925258799171844
the average of 10 rounds is 0.510248447204969
the average of 10 rounds is 0.533167701863354

Coherence Score:  0.34525539815002104

Perplexity:  -7.110665035217817

Coherence Score:  0.3525536134398247

Perplexity:  -7.155473393083183

Coherence Score:  0.323847535978391

Perplexity:  -7.1463372218509775

Coherence Score:  0.30352975127251436

Perplexity:  -7.1483151286707765
random forest classificaton:


...

the average of 10 rounds is 0.4783643892339544


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


the average of 10 rounds is 0.6025051759834369


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


the average of 10 rounds is 0.5709316770186336
complement naive bayes classifier:
the average of 10 rounds is 0.5345341614906832
the average of 10 rounds is 0.5043892339544513
the average of 10 rounds is 0.4941614906832298
the average of 10 rounds is 0.5348447204968944


In [55]:
# Create the TF-IDF model 
corpus_tfidf_1 = build_tfidf_model(corpus_1)
lda_tfidf_model_1 = build_LDA_model(dict_words_1, corpus_tfidf_1)
corpus_tfidf_2 = build_tfidf_model(corpus_2)
lda_tfidf_model_2 = build_LDA_model(dict_words_2, corpus_tfidf_2)
corpus_tfidf_3 = build_tfidf_model(corpus_3)
lda_tfidf_model_3 = build_LDA_model(dict_words_3, corpus_tfidf_3)
corpus_tfidf_4 = build_tfidf_model(corpus_4)
lda_tfidf_model_4 = build_LDA_model(dict_words_4, corpus_tfidf_4)

In [56]:
#Build LDA model
#NUM_TOPICS = 8
f = open('uneven_311_spotify_tfidf.txt', 'a')
for i in range(3, 8):
    NUM_TOPICS = i
    lda_model_1 = build_LDA_model(dict_words_1, corpus_tfidf_1)
    lda_model_2 = build_LDA_model(dict_words_2, corpus_tfidf_2)

    lda_model_3 = build_LDA_model(dict_words_3, corpus_tfidf_3)

    lda_model_4 = build_LDA_model(dict_words_4, corpus_tfidf_4)
    print("---------------------start of topic # {}".format(i), "-------------------", file = f)
    print_coherence_perplexity(lda_model_1, lemmatized_words_1, dict_words_1, corpus_tfidf_1)
    
    print_coherence_perplexity(lda_model_2, lemmatized_words_2, dict_words_2, corpus_tfidf_2)

    print_coherence_perplexity(lda_model_3, lemmatized_words_3, dict_words_3, corpus_tfidf_3)

    print_coherence_perplexity(lda_model_4, lemmatized_words_4, dict_words_4, corpus_tfidf_4)

    gen_df_mid_1 = generate_data(lda_model_1, corpus_tfidf_1)
    gen_df_mid_2 = generate_data(lda_model_2, corpus_tfidf_2)
    gen_df_mid_3 = generate_data(lda_model_3, corpus_tfidf_3)
    gen_df_mid_4 = generate_data(lda_model_4, corpus_tfidf_4)

    #randomForest_LDA(gen_df_mid_1, df_1.iloc[:TRAIN_SAMPLE, -1])

    #kfold for class 1:
    print("random forest classificaton:")
    rf_clf = RandomForestClassifier()
    rf_score_sum = kfold_classification(gen_df_mid_1, Y1, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_2, df_2.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 2:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_2, Y2, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_3, df_3.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 3:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_3, Y3, rf_clf, zscore_normalization)
    
    #randomForest_LDA(gen_df_mid_4, df_4.iloc[:TRAIN_SAMPLE, -1])
    #kfold for class 4:
    #random forest classificaton:
    rf_clf = RandomForestClassifier()
    rf_score_sum = rf_score_sum + kfold_classification(gen_df_mid_4, Y4, rf_clf, zscore_normalization)
    rf_score_avg = rf_score_sum / 4
    print("topics: {}, RF score average of 4 models: {}".format(i, rf_score_avg), file = f)

    print("complement naive bayes classifier:")
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = kfold_classification(gen_df_mid_1, Y1, cnb_clf)
    
    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_2, Y2, cnb_clf)

    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_3, Y3, cnb_clf)
    
    #complement naive bayes classifier:
    cnb_clf = ComplementNB(alpha= 0.1)
    cnb_score_sum = cnb_score_sum + kfold_classification(gen_df_mid_4, Y4, cnb_clf)
    cnb_score_avg = cnb_score_sum / 4
    print("Complement NB score average of 4 models: {}".format(cnb_score_avg), file = f)
    print("---------------------end of topic # {}".format(i), "-------------------", file = f)
    f.flush()
f.close()


Coherence Score:  0.3969742206535905

Perplexity:  -9.376353955368254

Coherence Score:  0.32138607973833483

Perplexity:  -9.657895332816478

Coherence Score:  0.46109291890740406

Perplexity:  -9.293907426335027

Coherence Score:  0.4832437873275315

Perplexity:  -9.157295351182563
random forest classificaton:
the average of 10 rounds is 0.5174534161490684


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


the average of 10 rounds is 0.5043478260869565
the average of 10 rounds is 0.4941407867494824


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


the average of 10 rounds is 0.498592132505176
complement naive bayes classifier:
the average of 10 rounds is 0.5159006211180124
the average of 10 rounds is 0.48559006211180133
the average of 10 rounds is 0.49569358178053824
the average of 10 rounds is 0.5101035196687371

Coherence Score:  0.3299494322518034

Perplexity:  -10.008204911499316

Coherence Score:  0.5924306586121187

Perplexity:  -9.43269441495482

Coherence Score:  0.3072325830485406

Perplexity:  -9.87950883656548

Coherence Score:  0.3056981433790496

Perplexity:  -9.898303364423473
random forest classificaton:


...

the average of 10 rounds is 0.4943478260869565


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


the average of 10 rounds is 0.5245755693581782


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


the average of 10 rounds is 0.46265010351966873
complement naive bayes classifier:
the average of 10 rounds is 0.5131055900621118
the average of 10 rounds is 0.5058178053830228
the average of 10 rounds is 0.4581573498964803
the average of 10 rounds is 0.504472049689441


In [20]:
#visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_1, corpus_1, dict_words_1)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.122696  0.002717       1        1  75.957802
2     -0.022563 -0.022634       2        1   6.834741
6      0.013202  0.038329       3        1   4.989365
5      0.021291 -0.006268       4        1   3.541031
1      0.021123 -0.000631       5        1   3.179031
0      0.054404 -0.006830       6        1   3.141583
4      0.035239 -0.004682       7        1   2.356454, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
47    Default  392.000000        wanna  392.000000  30.0000  30.0000
116   Default  401.000000         take  401.000000  29.0000  29.0000
531   Default  188.000000       really  188.000000  28.0000  28.0000
0     Default  552.000000         baby  552.000000  27.0000  27.0000
12    Default  464.000000          get  464.000000  26.0000  26.0000
249   Default  341.000000         girl  341.000000  25.0000  25.0000
14    Default  520.000000           go  520.000000  24.0000  24.0000
407   Default  171.000000         beat  171.000000  23.0000  23.0000
7     Default  422.000000         come  422.000000  22.0000  22.0000
31    Default  315.000000        night  315.000000  21.0000  21.0000
77    Default  234.000000       little  234.000000  20.0000  20.0000
382   Default  331.000000         away  331.000000  19.0000  19.0000
117   Default  316.000000        thing  316.000000  18.0000  18.0000
558   Default  135.000000         meet  135.000000  17.0000  17.0000
56    Default  150.000000         care  150.000000  16.0000  16.0000
257   Default  274.000000          lie  274.000000  15.0000  15.0000
211   Default  335.000000         back  335.000000  14.0000  14.0000
38    Default  294.000000          say  294.000000  13.0000  13.0000
1281  Default  146.000000         easy  146.000000  12.0000  12.0000
46    Default  385.000000         wait  385.000000  11.0000  11.0000
15    Default  333.000000         good  333.000000  10.0000  10.0000
475   Default  252.000000       enough  252.000000   9.0000   9.0000
103   Default  365.000000          see  365.000000   8.0000   8.0000
27    Default  409.000000         make  409.000000   7.0000   7.0000
259   Default  153.000000        round  153.000000   6.0000   6.0000
124   Default  599.000000         want  599.000000   5.0000   5.0000
86    Default  530.000000        never  530.000000   4.0000   4.0000
9     Default  442.000000         feel  442.000000   3.0000   3.0000
396   Default  345.000000         life  345.000000   2.0000   2.0000
1043  Default   65.000000          bit   65.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
661    Topic7    2.129016       dadada    9.240028   2.2801  -7.5590
2236   Topic7    2.714155       animal   11.786269   2.2796  -7.3162
5532   Topic7    1.939399      contain    8.422768   2.2795  -7.6523
718    Topic7    1.887536         gong    8.362846   2.2595  -7.6794
2473   Topic7    1.889013         slum    8.371264   2.2593  -7.6786
2030   Topic7    2.518049      ooooooh   11.189465   2.2565  -7.3912
660    Topic7    1.841372         dada    8.317986   2.2401  -7.7041
1554   Topic7    1.767224       detect    8.255151   2.2066  -7.7452
1807   Topic7    2.154755  unstoppable   10.104576   2.2027  -7.5470
1020   Topic7    3.644139         bill   18.736612   2.1107  -7.0215
2270   Topic7    3.982186        ocean   23.277729   1.9823  -6.9328
2636   Topic7    2.182436       barbie   10.285867   2.1977  -7.5342
2739   Topic7    2.206734         form   10.619337   2.1768  -7.5231
1      Topic7    2.776398         bang   21.036312   1.7229  -7.2935
1557   Topic7    3.114390        shape   30.615044   1.4626  -7.1786
2019   Topic7    4.012541        party   82.303688   0.7270  -6.9252
32     Topic7    3.990163          pay   83.168289   0.7110  -6.9308
302    Topic7    3.50

In [21]:
vis = pyLDAvis.gensim.prepare(lda_model_2, corpus_2, dict_words_2)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.188037 -0.000218       1        1  89.510323
2      0.028969  0.026899       2        1   2.579033
6      0.037969 -0.003282       3        1   2.044054
0      0.033507 -0.005582       4        1   1.647324
1      0.024458 -0.007288       5        1   1.538776
3      0.030831 -0.006729       6        1   1.369505
5      0.032304 -0.003800       7        1   1.310983, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
232   Default  189.000000      nobody  189.000000  30.0000  30.0000
928   Default   94.000000        past   94.000000  29.0000  29.0000
885   Default   99.000000         bet   99.000000  28.0000  28.0000
847   Default   61.000000       heavy   61.000000  27.0000  27.0000
254   Default  395.000000        head  395.000000  26.0000  26.0000
1188  Default   76.000000     thunder   76.000000  25.0000  25.0000
494   Default  144.000000     control  144.000000  24.0000  24.0000
574   Default  169.000000        kiss  169.000000  23.0000  23.0000
261   Default  465.000000        mind  465.000000  22.0000  22.0000
870   Default   84.000000       ahead   84.000000  21.0000  21.0000
1930  Default   98.000000        tree   98.000000  20.0000  20.0000
1698  Default   82.000000       shoot   82.000000  19.0000  19.0000
22    Default  282.000000        fire  282.000000  18.0000  18.0000
638   Default  197.000000    together  197.000000  17.0000  17.0000
746   Default  320.000000       alone  320.000000  16.0000  16.0000
734   Default   79.000000      family   79.000000  15.0000  15.0000
57    Default  568.000000        baby  568.000000  14.0000  14.0000
157   Default   89.000000        sick   89.000000  13.0000  13.0000
37    Default   54.000000      father   54.000000  12.0000  12.0000
209   Default  390.000000         run  390.000000  11.0000  11.0000
1248  Default  137.000000       carry  137.000000  10.0000  10.0000
872   Default  104.000000   beautiful  104.000000   9.0000   9.0000
342   Default  180.000000       bring  180.000000   8.0000   8.0000
1018  Default   45.000000       radio   45.000000   7.0000   7.0000
1757  Default  119.000000       share  119.000000   6.0000   6.0000
32    Default   62.000000     deserve   62.000000   5.0000   5.0000
1223  Default   84.000000         sea   84.000000   4.0000   4.0000
879   Default   32.000000         nah   32.000000   3.0000   3.0000
48    Default   64.000000        scar   64.000000   2.0000   2.0000
93    Default  554.000000        come  554.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
5447   Topic7    1.082834    hungover    5.539121   2.7021  -7.6045
2214   Topic7    1.150458     naughty    5.933657   2.6939  -7.5439
4850   Topic7    1.138198   temporary    6.106502   2.6545  -7.5546
1011   Topic7    1.009427  monopolize    5.432677   2.6513  -7.6747
8173   Topic7    0.998452  chandelier    5.456334   2.6361  -7.6856
1312   Topic7    0.984876   threshold    5.410306   2.6308  -7.6993
6672   Topic7    0.971019       papel    5.382070   2.6219  -7.7135
711    Topic7    1.577225       strip    8.847855   2.6099  -7.2284
6530   Topic7    1.073344         zag    6.039149   2.6069  -7.6133
453    Topic7    1.357436        ship    7.660353   2.6039  -7.3785
8521   Topic7    0.939214      likely    5.410839   2.5833  -7.7468
6953   Topic7    1.939768        toss   11.730232   2.5348  -7.0215
846    Topic7    2.080654      expose   12.768118   2.5201  -6.9514
6153   Topic7    1.216148       bayou    7.174744   2.5595  -7.4884
879    Topic7    3.608749         nah   32.491924   2.1368  -6.4007
1685   Topic7    1.153176        bark    6.732323   2.5700  -7.5415
847    Topic7    4.335969       heavy   61.390251   1.6841  -6.2171
1536   Topic7    1.417488        diss   10.205754   2.3603  -7.3352
334

In [22]:
vis = pyLDAvis.gensim.prepare(lda_model_3, corpus_3, dict_words_3)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.106367  0.004515       1        1  77.044395
0      0.037095 -0.014357       2        1   8.090554
6     -0.008652  0.025148       3        1   3.562343
2     -0.026131 -0.029933       4        1   3.461150
3     -0.040367  0.005194       5        1   2.857130
4     -0.023606  0.005646       6        1   2.799955
5     -0.044707  0.003787       7        1   2.184468, topic_info=     Category        Freq     Term       Total  loglift  logprob
term                                                            
304   Default  289.000000    could  289.000000  30.0000  30.0000
201   Default  234.000000      lie  234.000000  29.0000  29.0000
270   Default  336.000000     girl  336.000000  28.0000  28.0000
184   Default  423.000000     give  423.000000  27.0000  27.0000
89    Default  467.000000      get  467.000000  26.0000  26.0000
64    Default  287.000000     back  287.000000  25.0000  25.0000
14    Default  455.000000       go  455.000000  24.0000  24.0000
236   Default  454.000000     take  454.000000  23.0000  23.0000
4     Default  457.000000     come  457.000000  22.0000  22.0000
360   Default  438.000000     feel  438.000000  21.0000  21.0000
289   Default  591.000000     want  591.000000  20.0000  20.0000
250   Default  422.000000     baby  422.000000  19.0000  19.0000
221   Default  153.000000   reason  153.000000  18.0000  18.0000
95    Default  302.000000     home  302.000000  17.0000  17.0000
120   Default  311.000000    night  311.000000  16.0000  16.0000
72    Default  250.000000     call  250.000000  15.0000  15.0000
58    Default  195.000000    world  195.000000  14.0000  14.0000
25    Default  402.000000     make  402.000000  13.0000  13.0000
56    Default  391.000000      way  391.000000  12.0000  12.0000
91    Default  309.000000     good  309.000000  11.0000  11.0000
62    Default  244.000000   always  244.000000  10.0000  10.0000
644   Default  164.000000    sleep  164.000000   9.0000   9.0000
209   Default  497.000000    never  497.000000   8.0000   8.0000
282   Default  231.000000      run  231.000000   7.0000   7.0000
309   Default  236.000000    dream  236.000000   6.0000   6.0000
109   Default  268.000000     lose  268.000000   5.0000   5.0000
237   Default  392.000000     time  392.000000   4.0000   4.0000
2     Default  329.000000     away  329.000000   3.0000   3.0000
195   Default  328.000000     keep  328.000000   2.0000   2.0000
202   Default  231.000000     live  231.000000   1.0000   1.0000
...       ...         ...      ...         ...      ...      ...
7970   Topic7    1.742102     meat    8.192783   2.2756  -7.5746
584    Topic7    1.856271   barbie    8.780561   2.2698  -7.5111
434    Topic7    2.563699  science   12.715811   2.2224  -7.1883
542    Topic7    3.152079    knife   16.852011   2.1474  -6.9817
609    Topic7    7.062033  imagine   49.588211   1.8748  -6.1750
632    Topic7    4.825794  special   33.392330   1.8894  -6.5557
1491   Topic7    2.580507  monster   13.983437   2.1339  -7.1817
588    Topic7    3.263980    creep   23.535988   1.8482  -6.9468
15     Topic7    1.922574    greed    9.186036   2.2598  -7.4761
691    Topic7    2.827232     join   22.807348   1.7360  -7.0904
150    Topic7    2.480669     wife   18.263115   1.8274  -7.2212
1236   Topic7    2.491069    float   24.284138   1.5467  -7.2170
1241   Topic7    2.485400      let   24.418808   1.5389  -7.2193
58     Topic7    4.146328    world  195.312683  -0.0286  -6.7075
615    Topic7    3.920326     wish  198.770386  -0.1022  -6.7635
113    Topic7    3.474416      may  168.265350  -0.0563  -6.8843
180    Topic7    3.199656     fire  112.453278   0.2643  -6.9667
421    Topic7    3.439022     easy  165.100342  -0.0476  -6.8945
746    Topic7    2.968929   belong   84.279259   0.4779  -7.0415
887    Topic7    2.915164    water   77.041023   0.5494  -7.0598
689    Topic7    2.916081 

In [379]:
vis = pyLDAvis.gensim.prepare(lda_model_4, corpus_4, dict_words_4)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.049548 -0.031771       1        1  45.594128
1      0.035308  0.097619       2        1  23.136765
2      0.049721 -0.058243       3        1  17.371225
3     -0.134576 -0.007605       4        1  13.897882, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
834   Default   214.000000         que   214.000000  30.0000  30.0000
201   Default   889.000000        girl   889.000000  29.0000  29.0000
568   Default   272.000000         low   272.000000  28.0000  28.0000
34    Default  1543.000000        baby  1543.000000  27.0000  27.0000
1859  Default   239.000000         doo   239.000000  26.0000  26.0000
377   Default   542.000000       bitch   542.000000  25.0000  25.0000
1981  Default   226.000000         fly   226.000000  24.0000  24.0000
666   Default   498.000000        free   498.000000  23.0000  23.0000
25    Default  1436.000000        take  1436.000000  22.0000  22.0000
406   Default   586.000000        fuck   586.000000  21.0000  21.0000
171   Default   522.000000         put   522.000000  20.0000  20.0000
5     Default   923.000000       could   923.000000  19.0000  19.0000
244   Default   238.000000        wish   238.000000  18.0000  18.0000
1277  Default   471.000000        live   471.000000  17.0000  17.0000
347   Default  1207.000000       wanna  1207.000000  16.0000  16.0000
279   Default   363.000000        high   363.000000  15.0000  15.0000
220   Default   418.000000       nigga   418.000000  14.0000  14.0000
2116  Default   158.000000        lick   158.000000  13.0000  13.0000
257   Default   614.000000         run   614.000000  12.0000  12.0000
913   Default   511.000000       money   511.000000  11.0000  11.0000
657   Default   231.000000        cool   231.000000  10.0000  10.0000
2436  Default   182.000000      fallin   182.000000   9.0000   9.0000
479   Default   137.000000        bird   137.000000   8.0000   8.0000
524   Default   294.000000       stand   294.000000   7.0000   7.0000
134   Default   354.000000  everything   354.000000   6.0000   6.0000
41    Default   603.000000      little   603.000000   5.0000   5.0000
6     Default   950.000000        give   950.000000   4.0000   4.0000
1680  Default   152.000000       river   152.000000   3.0000   3.0000
250   Default   245.000000       happy   245.000000   2.0000   2.0000
982   Default   250.000000        rock   250.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
1917   Topic4    46.634361       super    63.011681   1.6725  -6.3088
657    Topic4   113.057495        cool   231.538055   1.2566  -5.4233
171    Topic4   173.166260         put   522.197876   0.8696  -4.9969
1680   Topic4    78.422691       river   152.286972   1.3098  -5.7890
1721   Topic4    46.177868        hole    65.606987   1.6223  -6.3187
377    Topic4   165.241180       bitch   542.133789   0.7853  -5.0437
1277   Topic4   151.780334        live   471.759644   0.8394  -5.1287
257    Topic4   173.084534         run   614.302979   0.7067  -4.9974
25     Topic4   282.343018        take  1436.532104   0.3466  -4.5080
1161   Topic4   100.689812        rain   252.478516   1.0542  -5.5391
5      Topic4   199.169235       could   923.290161   0.4396  -4.8570
18     Topic4   218.680420         see  1443.503418   0.0862  -4.7635
294    Topic4    97.632629         sun   305.911560   0.8313  -5.5699
347    Topic4   183.553421       wanna  1207.458862   0.0897  -4.9386
36     Topic4   213.430801        come  1772.607422  -0.1435  -4.7878
107    Topic4   186.786560        feel  1449.547363  -0.0756  -4.9212
94     Topic4   213.291595          go  1995.645630  -0.2626  -4.7885
40     Topic4   236.703110         get  2822.991211  -0.5053  -4.6843
220    Topic4    99.025597       nigga   418.502411   0.5321  -

model_1 = "model_1"

lda_model_1.save(model_1)

# print(corpus[0])
# for item in lda_model[corpus[0]]:
#     print("\n", item)

In [ ]:
# model_2 = "model_2"
# model_3 = "model_3"
# model_4 = "model_4"

# lda_model_2.save(model_2)
# lda_model_3.save(model_3)
# lda_model_4.save(model_4)

from gensim.test.utils import datapath

#save the model to disk
pretrained_model_1 = datapath("pretrained_model_1")
lda_model_1.save(pretrained_model_1)

#input new document to the LDA model, and see the score of classification

In [165]:
#view the topics in LDA model: the keyword in the num_topics topics
# from pprint import pprint
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

#load the pretrained model from disk
lda_model_1 = LdaModel.load(pretrained_model_1)

In [34]:
#evaluate the model with tf-idf
print_coherence_perplexity(lda_tfidf_model_1, lemmatized_words_1, dict_words_1, corpus_tfidf_1)


Coherence Score:  0.39085266637305865

Perplexity:  -8.858561083744949


In [ ]:
gen_df_tfidf_1 = generate_data(lda_tfidf_model_1, corpus_tfidf_1)
randomForest_LDA(gen_df_tfidf_1, df_1.iloc[:TRAIN_SAMPLE, -1])
print("Classify with Random Forest:")
rf_clf = RanfomForestClassifier()
kfold_classification(gen_df_tfidf_1, Y1, rf_clf)
print("Classify with Complement Naive Bayes classifier:")
cnb_clf = ComplementNB(alpha = 0.1)
kfold_classification(gen_df_tfidf_1, Y1, cnb_clf)

## As we can see that after applying tf-idf model as the corpus input, the cohenrence score increased by 0.01, while the classification score decreased by 0.15.

In [169]:
from sklearn.linear_model import LassoCV
#it's said the best model will be chosen by cross-validation


## Improve upon this model by using Mallet's version of LDA algorithm, which always gives a better quality of topics than Gensim's inbuilt version of the LDA algorithm.

In [38]:
#use mallet's implementation of LDA
mallet_path = "../mallet-2.0.8/bin/mallet"
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_1, num_topics=NUM_TOPICS, id2word=dict_words_1)
# evaluate the mallet model: Compute Coherence Score
coherence_model = CoherenceModel(model = ldamallet, texts = lemmatized_words_1, dictionary = dict_words_1, coherence = 'c_v')
coherence_lda = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3783316243882765


#classify the dataset using output of mallet's LDA model
gen_df_mallet_1 = generate_data(ldamallet, corpus_1)
randomForest_LDA(gen_df_mallet_1, df_1.iloc[:TRAIN_SAMPLE, -1])
print("Classfy using data generated by Mallet's LDA with Random Forest Classifier:")
## AttributeError: 'LdaMallet' object has no attribute 'get_document_topics'

In [40]:
#try tf-idf on mallet
ldamallet_tfidf = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_tfidf_1, num_topics=NUM_TOPICS, id2word=dict_words_1)
# evaluate the mallet model: Compute Coherence Score
coherence_model = CoherenceModel(model = ldamallet_tfidf, texts = lemmatized_words_1, dictionary = dict_words_1, coherence = 'c_v')
coherence_lda = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.7/site-packages/gensim/models/wrappers/ldamallet.py:380: RuntimeWarning: invalid value encountered in true_divide
  return topics / topics.sum(axis=1)[:, None]



Coherence Score:  0.45514237865985774


How to infer pyLDAvis's output?

Each bubble represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles, clustered in one region of the chart

So this model ... maybe use fewer number of topics

You could determine the number of topics using prior knowledge of the number of natural topics in the document

In [ ]:
#We will focus on how to arrive at the optimal number of topics given any large corpus of text
#TODO: I really want to know what can gridsearch do in finding the optimal number of topics

In [ ]:
# Show the TF-IDF weights
for doc in tfidf[corpus]:
    print([[dict_words[id], np.around(freq, decimals=2)] for id, freq in doc])